https://www.geeksforgeeks.org/how-to-extract-images-from-pdf-in-python/

In [ ]:
!pip install fitz

In [4]:
!pip install frontend

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

  Using cached asgiref-3.5.0-py3-none-any.whl (22 kB)
  Attempting uninstall: asgiref
    Found existing installation: asgiref 3.3.1
    Uninstalling asgiref-3.3.1:
      Successfully uninstalled asgiref-3.3.1
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0



You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [26]:
import fitz
import io

In [67]:
# file = "Book2.pdf"
# file = "Book1.pdf"
file = "abc.pdf"

# file = "Atlantic_2022.02.02.pdf"

In [69]:
file = r'c:\temp\abc.pdf'

In [70]:
pdf_file = fitz.open(file)

In [17]:
for page_index in range(len(pdf_file)):
    
    # get the page itself
    page = pdf_file[page_index]
    image_list = page.get_images()
      
    # printing number of images found in this page
    if image_list:
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    else:
        print("[!] No images found on page", page_index)
    for image_index, img in enumerate(page.get_images(), start=1):
        
        # get the XREF of the image
        xref = img[0]
          
        # extract the image bytes
        base_image = pdf_file.extractImage(xref)
        image_bytes = base_image["image"]
          
        # get the image extension
        image_ext = base_image["ext"]

[!] No images found on page 0


In [71]:
len(pdf_file)

1

In [72]:
page = pdf_file[0]

In [83]:
with open('abc.svg', 'wt') as f:
    f.write(page.get_svg_image())

In [84]:
from pathlib import Path

In [86]:
svg_path = Path(r"abc.svg")


In [87]:
svg_path.is_file()

True

In [88]:
import re

In [89]:
with open(svg_path, 'rt', encoding='utf-8') as f:
    sz = f.read()

In [216]:
# with open('abc.txt', 'wt') as fw:
        
x_max = 0
x_min = 99999
y_max = 0
y_min = 99999

for mtch in re.finditer('\n<path transform.*? d="(.*?)"', sz):
        
    s = mtch.group(1)
    # print(s)
    for mtch2 in re.finditer('[L|M] (\S+) (\S+)', s):
        try:
            x = float(mtch2.group(1))
            y = float(mtch2.group(2))
        except ValueError:
            print(s, mtch2.group(1), mtch2.group(2))
            raise Exception
            
        if x < x_min:
            x_min = x
        if x > x_max:
            x_max = x
        if y < y_min:
            y_min = y
        if y > y_max:
            y_max = y

print(x_min, x_max, y_min, y_max)
    # print(float(x_max)-float(x_min), float(y_max)-float(y_min))

50.4 791.526 83.04 512.27999


In [107]:
re.search(r'"matrix\(.*,(.*)\)"', sz)

<re.Match object; span=(414, 441), match='"matrix(1,0,0,-1,0,595.32)"'>

In [91]:
mtch = re.search(r'"matrix\(.*,(.*)\)"', sz).group(1)
    y_edge = float(m.group(1))
    break
print(y_edge)

595.32


In [92]:
y_max

512.27999

In [93]:
viewBox= f'{x_min:.2f} {y_edge-y_max:.2f} {x_max-x_min:.2f} {y_max-y_min:.2f}'
print(viewBox)

50.40 83.04 741.13 429.24


In [94]:
sz2 = re.sub(r'viewBox=".*?"', f'viewBox="{viewBox}"', sz  )

In [95]:
sz2 = re.sub(r' width=".*?" height=".*?"', '', sz2)

In [96]:
with open('abcdef.svg','wt', encoding='utf-8') as f:
    f.write(sz2)

In [45]:
y_max

787.07998

In [98]:
pdf_path = Path(r'c:\temp\abc.pdf')

In [102]:
svg_path = pdf_path.with_suffix('.svg')

In [103]:
svg_path

WindowsPath('c:/temp/abc.svg')

In [222]:
def reduce_svg_viewbox(svg: str) -> str:
    
    '''Removes the padding around an svg that has been extracted from a pdf exporteded from Excel

    '''
    Returns:    the svg string with the padding removed

    x_max = 0
    x_min = 99999
    y_max = 0
    y_min = 99999

    for mtch in re.finditer('\n<path transform.*? d="(.*?)"', svg):
            
        s = mtch.group(1)
        # print(s)
        # for i, mtch2 in enumerate(re.finditer('[A-Z] (\S+) (\S+)', s)):
        # print(s)
        for i, mtch2 in enumerate(re.finditer('[L|M] (\S+) (\S+)', s)):    
            try:
                x = float(mtch2.group(1))
                y = float(mtch2.group(2))
            except ValueError:
                print(s, mtch2.group(1), mtch2.group(2))
                raise Exception
            if x < x_min:
                x_min = x
            if x > x_max:
                x_max = x
            if y < y_min:
                y_min = y
            if y > y_max:
                y_max = y
        
        if i == 5:     
            break # break out after the first match - seems to work!!! 
    
    y_edge = re.search(r'"matrix\(.*,(.*)\)"', svg).group(1) # multiple matches but all the same: take first one
    y_edge = float(y_edge)
    
    print(f'{x_min=}, {x_max=}, {y_min=}, {y_max=}, {y_edge=}')

    reduced_viewbox_string= f'{x_min:.2f} {y_edge-y_max:.2f} {x_max-x_min:.2f} {y_max-y_min:.2f}'
    print(reduced_viewbox_string)
    svg = re.sub(r'viewBox=".*?"', f'viewBox="{reduced_viewbox_string}"', svg, count=1) # sub first match at top only

    # remove width and height
    svg = re.sub(r' width=".*?" height=".*?"', '', svg, count=1)

    return svg

In [229]:
def svg_from_excel_pdf(pdf_path: Path) -> int:
    
    '''Extracts the svg from a single-page pdf, removes the padding around the svg and and saves it to a file
    with the same name as the pdf 
    '''
    
    svg_path = pdf_path.with_suffix('.svg')
    
    with fitz.open(pdf_path) as pdf_file:        
        svg = pdf_file[0].get_svg_image()
    
    svg = reduce_svg_viewbox(svg)
    
    with open(svg_path, 'wt') as svg_file:
        svg_file.write(svg)
    
    return 0

In [227]:
pie = Path(r'c:\temp\pienew.pdf')

In [230]:
svg_from_excel_pdf(pie)

x_min=50.4, x_max=791.52, y_min=82.8, y_max=512.64, y_edge=595.32
50.40 82.68 741.12 429.84


0

In [184]:
595.32 - 511.782

83.53800000000007

In [143]:
790.872 - 51.172

739.6999999999999

In [144]:
511.78199 - 83.532

428.24999

This header works fine:
```Code
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" viewBox="51.17 83.54 739.70 428.25">